Create frames for animation similar to `swim_surfing_v0.mp4` but using the output from 
2024-03 runs in `run_dfm_bloom_tracers_v02.ipynb`

These runs have a uniform release, although variation in depth and vertical migration
create horizontal variation in concentration, in addition to the vertical migration.

Aside from targeting a different run, this code also switches to processing DFM output
since recent runs are online coupled.

2024-05-28: Updating to the v08 run.
2024-09-27: Update to 6.912(j) with temperature, v12(?)

In [1]:
import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
import sys
sys.path.append("/richmondvol1/rusty/stompy")
from stompy import utils
import stompy.model.delft.dflow_model as dfm
import stompy.model.delft.waq_scenario as dwaq
from stompy.grid import unstructured_grid
from matplotlib import colors
import os
import numpy as np
import xarray as xr
from stompy.plot import mkanim

from matplotlib import colors

from stompy import xr_transect, xr_utils

import subprocess
from shapely import wkt

from IPython.display import Video
import six
import glob
from stompy.plot import nbviz, plot_wkb

In [3]:
#run_dir='bloom_tracers_v04/run_20220802T1200_20220830T0000_v02'
run_dir='bloom_tracers_v08/run_20220802T1930_20220830T0000_v00'

In [4]:
model = dfm.DFlowModel.load(run_dir)
ds=model.map_dataset(chain=False)

In [5]:
g=ds.grid

In [6]:
zoom=(525166.5357195626, 595636.7532705697, 4147408.0927562183, 4202218.261962557)

In [7]:
# line from W span of bay bridge to about Redwood City
profile="""
LineString (553934.46602910873480141 4185292.98902965523302555, 556003.0745520245982334 4182475.40155878709629178, 557144.37580604711547494 4180335.46170749515295029, 557964.68608237581793219 4178231.18752039084210992, 558250.01139588153455406 4176198.24466166319325566, 558606.66803776356391609 4172631.67824284266680479, 559248.64999315119348466 4169279.10580915166065097, 560211.62292623275425285 4166283.19001734210178256, 561994.90613564301747829 4163750.92785997968167067, 563706.85801667685154825 4161896.31332219298928976, 565882.46353215735871345 4159970.36745603010058403, 568129.40037601429503411 4158650.73788106627762318, 571089.65050363529007882 4156831.78900746814906597, 573728.90965356235392392 4154406.52384267002344131)
"""
profile_geom_channel=wkt.loads(profile) 


profile_shoal="""
LineString (553943.43526522838510573 4185306.34642233327031136, 555905.91317636473104358 4182700.43313049618154764, 557064.09686162555590272 4180319.72222190443426371, 557932.7346255712909624 4178099.8701584879308939, 558318.79585399152711034 4175686.98748086113482714, 558447.48293013160582632 4174046.2272600750438869, 558994.40300372685305774 4171392.05631468491628766, 560184.75845802284311503 4169477.83605710230767727, 562565.46936661447398365 4168030.10645052650943398, 570930.12931572052184492 4165456.3649277244694531, 574211.64975729293655604 4164716.4142399188131094)
"""
# similar northern run, but in the middle of San Bruno Shoal head out across the shoal
profile_geom_shoal=wkt.loads(profile_shoal)


if 1:
    profile_name="channel"
    profile_geom=profile_geom_channel
    
if 0:
    profile_name='shoal'
    profile_geom=profile_geom_shoal


In [8]:
# Grab output at single cell, see how vertical distribution is changing
tracers=['conc1']
tracer_names=['10']

cells,dists=g.select_cells_intersecting(profile_geom,order=True,return_distance=True)
cc=g.cells_center()

c_ref=cells[ len(cells)//2]

#depth_ref=ds['TotalDepth'].isel(layer=0,face=c_ref)

In [9]:
tidx=10

def extract_transect(tidx):
    tran=xr.Dataset()
    snap = ds.isel(time=tidx)

    # DFM map output we have to reconstruct sigma...
    tran['LayCoord_cc'] = snap['LayCoord_cc']
    tran['LayCoord_w'] = snap['LayCoord_w']
    # inputs for sigma transform
    tran['s1'] = snap['s1'].isel(nFlowElem=cells)
    tran['FlowElem_bl'] = snap['FlowElem_bl'].isel(nFlowElem=cells)

    tran['z_ctr'] = xr_utils.decode_sigma(tran, tran.LayCoord_cc)
    tran['z_int'] = xr_utils.decode_sigma(tran, tran.LayCoord_w)

    tran = tran.set_coords(['LayCoord_cc','LayCoord_w','z_ctr','z_int'])
    
    #tran['z_int']=('sample','interface'),interfaces
    #tran['z_ctr']=('sample','layer'), 0.5*(interfaces[:,:-1] + interfaces[:,1:])

    for tracer in tracers: 
        tran[tracer ] = snap[tracer].isel(nFlowElem=cells)
        # tran[tracer]=('sample','layer'),conc.transpose('nFlowElem','laydim').values

    tran = tran.rename({'nFlowElem':'sample', 'laydim':'layer', 'wdim':'interface'})
    tran['d_sample']=('sample',),dists
    tran['cell']=('sample',),cells
    tran['x_sample']=cc[cells,0]
    tran['y_sample']=cc[cells,1]

    return tran

tran= extract_transect(10)

In [10]:
depth_ref = ds['s1'].isel(nFlowElem=cells[0])

In [11]:
save=True

tracer='conc1'
tracer_name='10mpd'

if save:
    frame_dir=os.path.join(run_dir,f'swim_frames_{profile_name}_{tracer_name}_v0')
    if not os.path.exists(frame_dir):
        os.makedirs(frame_dir)
    [os.unlink(f) for f in glob.glob(os.path.join(frame_dir,'*.png'))]

    patt=os.path.join(frame_dir,'frame_%04d.png')

    plt.ioff()
    tidxs=range(ds.dims['time'])
else:
    tidxs=[-1]

normalize = True

for tidx in tidxs:
    print(tidx)
    tran=extract_transect(tidx)

    if normalize: # normalize depth-averaged concentration.
        tran[tracer] /= tran[tracer].mean(dim='layer')

    fig,axs=plt.subplots(len(tracers)+1,1,figsize=(8.5,5), height_ratios=[0.7,0.3])
    ax_conc=None # axs[-2]
    ax_t=axs[-1]
    ax_d=ax_t.twinx()

    ccolls=[]
    for tracer,name,ax in zip(tracers,tracer_names,axs):
        ccoll=xr_transect.plot_scalar(tran, tracer, ax=ax,cmap='turbo')
        ccolls.append(ccoll)
        if ax_conc is not None:
            ax_conc.plot(tran.d_sample, tran[tracer].mean(dim='layer'),label=name)

    if normalize:
        for coll in ccolls:
            coll.set_clim([0.5,2.0] )
    elif 1: # match color limits
        for coll in ccolls[:-1]:
            coll.set_norm( ccolls[-1].norm )
        
    ax_d.plot(ds.time, depth_ref)
    ax_d.axvline(ds.time.values[tidx],color='k')

    dnum_pdt=utils.to_dnum( ds.time.values-np.timedelta64(7,'h'))
    daylight=(-np.cos(dnum_pdt*2*np.pi)).clip(0)

    ax_t.fill_between(ds.time,daylight,zorder=-1,fc='yellow',ec='k',lw=0.5)

    for ax in axs[:-1]:
        ax.axis(xmin=tran.d_sample.values[0],xmax=tran.d_sample.values[-1],ymin=-34,ymax=3.0)
    ax_t.axis(ymin=0,xmin=ds.time.values[0], xmax=ds.time.values[-1])
    ax_t.yaxis.set_visible(0)

    if ax_conc is not None:
        ax_conc.legend(loc='upper left',bbox_to_anchor=[1.,1],frameon=0,title='w$_s$ (m/d)')
        ax_conc.text(0.01,0.98,"Depth avg.",transform=ax_conc.transAxes, va='top')
    
    fig.autofmt_xdate()
    fig.subplots_adjust(top=0.98,right=0.85,left=0.06,bottom=0.12)
    
    cax=fig.add_axes( [0.89, axs[len(tracers)-1].get_position().ymin, 0.02,
                       axs[0].get_position().ymax - axs[len(tracers)-1].get_position().ymin] )
    plt.colorbar(ccolls[-1], cax=cax)

    if save:
        fig.savefig(patt%tidx)
        plt.close(fig)

if save:
    plt.ion()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
swim_mp4=os.path.join(run_dir,f'swim_surfing_{profile_name}_{tracer_name}_v0.mp4')
mkanim.mkanim(swim_mp4,patt,fps=12)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [15]:
slow_swim_mp4=os.path.join(run_dir,f'swim_surfing_{profile_name}_{tracer_name}_6fps_v0.mp4')
mkanim.mkanim(slow_swim_mp4,patt,fps=6)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [17]:
Video(slow_swim_mp4)

In [14]:
swim_mp4

'bloom_tracers_v08/run_20220802T1930_20220830T0000_v00/swim_surfing_channel_10mpd_v0.mp4'

In [68]:
plt.close('all')